# *tridesclous* example with locust dataset

Here a detail notebook that detail the locust dataset recodring by Christophe Pouzat.

This dataset is our classic.
It has be analyse yet by several tools in R, Python or C:
  * https://github.com/christophe-pouzat/PouzatDetorakisEuroScipy2014
  * https://github.com/christophe-pouzat/SortingABigDataSetWithPython
  * http://xtof.perso.math.cnrs.fr/locust.html

So we can compare the result.

The original datasets is here https://zenodo.org/record/21589

But we will work on a very small subset on github https://github.com/tridesclous/tridesclous_datasets/tree/master/locust


# Overview

In *tridesclous*, the spike sorting is done in several step:
  * Define the datasource and working path. (class DataIO)
  * Construct a *catalogue* (class CatalogueConstructor) on a short chunk of data (for instance 60s)
    with several sub step :
    * signal pre-processing:
      * high pass filter (optional)
      * removal of common reference (optional)
      * noise estimation (median/mad) on a small chunk
      * normalisation = robust z-score
    * peak detection
    * extract some waveform. Unecessary and impossible to extract them all.
    * find rational limit of waveforms (n_left/n_right)
    * project theses waveforms in smaller dimention (pca, ...)
    * find cluster
    * clean with GUI (class CatalogueWindow)
    * save centroids (median+mad + first and second derivative)
  * Apply the *Peeler* (class Peeler) on the long term signals. With several sub steps:
     * same signal preprocessing than before
     * find peaks
     * find the best cluster in catalogue for each peak
     * find the intersample jitter
     * remove the oversampled waveforms from the signals until there are not peaks in the signals.
     * check with GUI (class PeelerWindow)



In [1]:
%matplotlib inline

import time
import numpy as np
import matplotlib.pyplot as plt
import tridesclous as tdc

from tridesclous import DataIO, CatalogueConstructor, Peeler

# Download a small dataset

trideclous provide some datasets than can be downloaded with **download_dataset**.

Note this dataset contains 2 trials in 2 different files. (the original contains more!)

Each file is considers as a *segment*. *tridesclous* automatically deal with it.

In [8]:
#download dataset
localdir, filenames, params = tdc.download_dataset(name='locust')
print(filenames)
print(params)

['/home/samuel/Documents/projet/tridesclous/example/locust/locust_trial_01.raw', '/home/samuel/Documents/projet/tridesclous/example/locust/locust_trial_02.raw']
{'total_channel': 4, 'sample_rate': 15000.0, 'dtype': 'int16'}


# DataIO = define datasource and working dir


Theses 2 files are in **RawData** format this means binary format with interleaved channels.

Our dataset contains 2 segment of 28.8 second each, 4 channels. The sample rate is 15kHz.

In [11]:
#create a DataIO
import os, shutil
dirname = 'tridesclous_locust'
if os.path.exists(dirname):
    #remove is already exists
    shutil.rmtree(dirname)    
dataio = DataIO(dirname=dirname)

# feed DataIO
dataio.set_data_source(type='RawData', filenames=filenames, **params)

print(dataio)

    
    

DataIO <id: 140599959595664> 
  workdir: tridesclous_locust
  sample_rate: 15000.0
  total_channel: 4
  channel_groups: 0 [0 1 2 3]
  nb_segment: 2
  length: 431548 431548
  durations: 28.8 28.8 s.


# CatalogueConstructor

In [12]:
catalogueconstructor = CatalogueConstructor(dataio=dataio)
print(catalogueconstructor)

CatalogueConstructor <id: 140599959472168> 
  workdir: tridesclous_locust/channel_group_0/catalogue_constructor
  Signal pre-processing not done yet


## Set some parameters for the pre-processing step.

For a complet description of each params see main documentation.

In [13]:
catalogueconstructor.set_preprocessor_params(chunksize=1024,
            common_ref_removal=False,
            highpass_freq=300.,
            backward_chunksize=1280,
            peak_sign='-',
            relative_threshold=7,
            peak_span=0.0005,
            )

## Estimate the noise on a small chunk of filtered signals.
This compute medians and mad of each channel.

In [14]:
catalogueconstructor.estimate_signals_noise(seg_num=0, duration=15.)
print(catalogueconstructor.signals_medians)
print(catalogueconstructor.signals_mads)

[ 1.33329058  0.97232521  1.79204416  0.52885079]
[ 57.52484894  52.35477066  63.70387268  51.0644455 ]


## Run the main loop: signal preprocessing + peak detection



In [15]:
t1 = time.perf_counter()
catalogueconstructor.run_signalprocessor(duration=60.)
t2 = time.perf_counter()

print('run_signalprocessor', t2-t1, 's')
print(catalogueconstructor)

run_signalprocessor 0.7115417329987395 s
CatalogueConstructor <id: 140599959472168> 
  workdir: tridesclous_locust/channel_group_0/catalogue_constructor
  nb_peak: 1126
  nb_peak_by_segment: 552, 574
  cluster_labels [-10]



## extract some waveforms

Take some waveforms in the signals *n_left/n_right* must be choosen arbitrary but lon enought.
Better limits will be set later.

In [17]:
catalogueconstructor.extract_some_waveforms(n_left=-25, n_right=40, mode='rand', nb_max=10000, align_waveform=True)
print(catalogueconstructor)

CatalogueConstructor <id: 140599959472168> 
  workdir: tridesclous_locust/channel_group_0/catalogue_constructor
  nb_peak: 1126
  nb_peak_by_segment: 552, 574
  n_left -25 n_right 40
  some_waveforms.shape: (1126, 65, 4)
  cluster_labels [-10]



## Find good limits for waveforms and re-extract

To avoid useless portion of signal on the sides of peaks we take smaller sweep.
This technics is based on the MAD. We take only central zone where the MAD is above the noise.
Noise is 1. In practice we take a bit more 1.1

Here the methods give a "good limts" of n_left -10 n_right 15.

So the shape of waveforms become smaller.




In [18]:
n_left, n_right = catalogueconstructor.find_good_limits(mad_threshold = 1.1,)
print(catalogueconstructor)


CatalogueConstructor <id: 140599959472168> 
  workdir: tridesclous_locust/channel_group_0/catalogue_constructor
  nb_peak: 1126
  nb_peak_by_segment: 552, 574
  n_left -10 n_right 13
  some_waveforms.shape: (1126, 23, 4)
  cluster_labels [-10]



## Project to smaller space

In [19]:
t1 = time.perf_counter()
catalogueconstructor.project(method='pca', n_components=5)
t2 = time.perf_counter()
print('project', t2-t1)
print(catalogueconstructor)

project 0.05169410499911464
CatalogueConstructor <id: 140599959472168> 
  workdir: tridesclous_locust/channel_group_0/catalogue_constructor
  nb_peak: 1126
  nb_peak_by_segment: 552, 574
  n_left -10 n_right 13
  some_waveforms.shape: (1126, 23, 4)
  some_features.shape: (1126, 5)
  cluster_labels [-10]



# find clusters

In [20]:
t1 = time.perf_counter()
catalogueconstructor.find_clusters(method='kmeans', n_clusters=12)
t2 = time.perf_counter()
print('find_clusters', t2-t1)
print(catalogueconstructor)


find_clusters 0.16290489899984095
CatalogueConstructor <id: 140599959472168> 
  workdir: tridesclous_locust/channel_group_0/catalogue_constructor
  nb_peak: 1126
  nb_peak_by_segment: 552, 574
  n_left -10 n_right 13
  some_waveforms.shape: (1126, 23, 4)
  some_features.shape: (1126, 5)
  cluster_labels [ 0  1  2  3  4  5  6  7  8  9 10 11]



## Open CatalogueWindow for visual check

At the end we can save the catalogue.

In [21]:
%gui qt5
import pyqtgraph as pg
app = pg.mkQApp()
win = tdc.CatalogueWindow(catalogueconstructor)
win.show()
app.exec_()    

construct_catalogue 0.11879534400031844


0

Here a snappshot of CatalogueWindow

<img src="../doc/img/snapshot_cataloguewindow.png">


In [22]:
catalogueconstructor.save_catalogue()

construct_catalogue 0.13998361099947942


# Peeler

Create and run the Peeler.
It should be pretty fast, here the computation take 1.32s for 28.8x2s of signal. This is a speed up of 43 over real time.


In [23]:
initial_catalogue = dataio.load_catalogue(chan_grp=0)

peeler = Peeler(dataio)
peeler.change_params(catalogue=initial_catalogue, n_peel_level=2)

t1 = time.perf_counter()
peeler.run()
t2 = time.perf_counter()
print('peeler.run', t2-t1)

print()
for seg_num in range(dataio.nb_segment):
    spikes = dataio.get_spikes(seg_num)
    print('seg_num', seg_num, 'nb_spikes', spikes.size)
    


peeler.run 1.3970982890004962

seg_num 0 nb_spikes 568
seg_num 1 nb_spikes 582


## Open PeelerWindow for visual checking

In [24]:
%gui qt5
import pyqtgraph as pg
app = pg.mkQApp()
win = tdc.PeelerWindow(dataio=dataio, catalogue=initial_catalogue)
win.show()
app.exec_()


-1

Here a snappshot of PeelerWindow

<img src="../doc/img/snapshot_peelerwindow.png">